## LangChain
- 자연어 처리(NLP)와 생성형 AI 응용 프로그램을 개발하기 위한 프레임워크입니다. 주로 대형 언어 모델(LLMs)과 같은 최신 NLP 기술을 기반으로 하여 다양한 작업을 자동화하거나 개선할 수 있는 도구와 서비스를 제공합니다.
- 이 프레임워크는 특히 언어 모델의 기능을 확장하고 이를 보다 쉽게 사용할 수 있도록 설계되었습니다.
- LangChain의 핵심 목표는 언어 모델을 활용하여 여러 복잡한 작업을 수행할 수 있도록 돕는 것이며, 특히 긴 텍스트, 문서 체인 또는 여러 단계의 워크플로를 필요로 하는 복잡한 응용 프로그램에 유용합니다.

### LangChain의 구성 요소:
- Language Models: 언어 모델 자체를 사용하여 텍스트 생성, 요약, 번역 등의 작업을 수행합니다.
- Chains: 여러 모델 호출을 연결하여 복잡한 작업을 수행하는 논리적 흐름을 정의할 수 있습니다. 예를 들어, 텍스트를 요약한 후 요약된 텍스트에 대한 질의응답을 수행하는 체인을 만들 수 있습니다.
- Agents: 주어진 작업에 따라 다양한 툴을 선택하고 사용할 수 있는 지능형 에이전트입니다. 예를 들어, 정보 검색, API 호출 등을 수행하는 역할을 합니다.
- Memory: 이전의 상호작용 또는 맥락을 기억하는 기능입니다. 이를 통해 대화형 AI나 컨텍스트를 유지해야 하는 애플리케이션을 구현할 수 있습니다.

### 활용 사례:
- 대화형 에이전트: 사용자와의 대화에서 맥락을 유지하며 대답할 수 있는 챗봇 개발.
문서 처리: 긴 문서나 여러 문서의 내용을 요약하거나 분석하는 애플리케이션.
- 지식 탐색: 사용자가 특정 주제에 대해 질문하면, 관련된 정보를 검색하고 이를 바탕으로 대답을 제공하는 시스템.

### 통합:
- LangChain은 다양한 데이터 소스, API 및 언어 모델과 통합될 수 있으며, 이를 통해 다양한 도메인에서 사용될 수 있습니다. 예를 들어, SQL 데이터베이스에 쿼리를 보내고 결과를 요약하거나, 웹에서 정보를 수집한 후 이를 기반으로 질문에 답하는 작업을 수행할 수 있습니다.

이 프레임워크는 특히 연구자, 데이터 사이언티스트, 개발자들이 생성형 AI를 활용하여 복잡한 텍스트 기반 응용 프로그램을 구축하는 데 큰 도움이 됩니다. LangChain은 이러한 작업을 더 쉽게, 더 직관적으로 구현할 수 있게 도와줍니다.

### LangChain 패키지를 사용하여 RAG 시스템을 구성하는 예제
주어진 질문에 대해 텍스트 파일에 저장된 문서에서 관련 정보를 검색하고, 이를 기반으로 GPT 모델이 답변을 생성하는 것입니다.

- langchain-openai: LangChain과 OpenAI를 연결하는 패키지.
- faiss-cpu: 벡터 검색을 위한 FAISS 라이브러리의 CPU 버전.
- langchain_community: LangChain의 커뮤니티에서 개발된 추가 도구.
- tiktoken: OpenAI 모델에서 사용하는 토큰화 도구.

In [5]:
!pip install -q openai
!pip install langchain langchain-openai -q
!pip install faiss-cpu -q
!pip install langchain_community -q
!pip install tiktoken -q

In [6]:
from openai import OpenAI

## Set the API key and model name
MODEL="gpt-4o-mini-2024-07-18"
client = OpenAI(api_key="sk-proj-upOVy97VapEQl5kOiTQIT3BlbkFJlKBrtAnjDU5YMvWrqMzd")

documents.txt

In [9]:
# Define the content for both basic and advanced sections
content = """
Document 1:
Polar bears are directly impacted by climate change due to their dependence on sea ice for hunting seals. As global temperatures rise, sea ice is melting earlier and forming later each year, reducing the time polar bears have to hunt. This has led to decreased body condition, lower cub survival rates, and in some cases, increased mortality.

Document 2:
The loss of sea ice habitat is one of the most significant threats to polar bear populations. As the ice retreats, polar bears are forced to travel greater distances to find food, leading to increased energy expenditure. This can result in malnutrition and a decline in reproductive success, further endangering the species.

Document 3:
Climate change is not only affecting polar bears' hunting grounds but also their denning areas. Warmer temperatures and unstable snow conditions are making it more difficult for pregnant females to find suitable denning sites, which is crucial for the birth and survival of their cubs. This adds an additional layer of risk to polar bear populations already struggling due to loss of sea ice.

Document 4:
As their traditional food sources become less accessible, some polar bears have been observed scavenging on human waste or preying on seabirds and their eggs. While this behavior may provide some short-term relief, it does not replace the high-fat diet they obtain from seals, which is essential for their long-term survival in the harsh Arctic environment.

Document 5:
Recent studies suggest that if current trends in greenhouse gas emissions continue, polar bears could face extinction within this century. Conservation efforts are focusing on reducing global emissions and protecting critical polar bear habitats, but these efforts may not be sufficient if the climate continues to warm at the current rate.
"""

# Write the content to a file
with open('documents.txt', 'w') as file:
    file.write(content)

In [10]:
!ls

documents.txt  sample_data


In [ ]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA

- OpenAIEmbeddings: 문서를 벡터로 임베딩하는 클래스입니다. 각 문서를 숫자 벡터로 변환하여 유사성 검색을 할 수 있게 만듭니다.
- ChatOpenAI: OpenAI의 GPT 모델을 사용하는 클래스입니다. model을 지정하여 특정 GPT 모델을 사용할 수 있으며, fine-tuned된 모델도 지원합니다.
- FAISS: 벡터를 빠르게 검색하기 위한 라이브러리입니다. Facebook에서 개발한 AI 유사성 검색 라이브러리로, 임베딩된 문서들 중에서 가장 관련성이 높은 문서를 빠르게 검색할 수 있습니다.
- TextLoader: 문서를 텍스트 형식으로 불러오는 클래스입니다. 이 클래스는 주어진 파일에서 텍스트 데이터를 로드할 때 사용됩니다.
- RetrievalQA: 검색된 문서와 GPT 모델을 결합하여 질의응답 작업을 수행하는 체인입니다.

In [ ]:
# OpenAI API 키 설정
import os
os.environ["OPENAI_API_KEY"] = 'sk-proj-upOVy97VapEQl5kOiTQIT3BlbkFJlKBrtAnjDU5YMvWrqMzd'  # OpenAI API 키 입력

# 1. 문서 로드
# 텍스트 파일을 사용해 문서를 로드합니다.
loader = TextLoader('documents.txt')
documents = loader.load()

이 코드는 문서를 로드하는 부분입니다.

- **TextLoader**를 사용하여 documents.txt 파일을 불러옵니다. 이 파일은 시스템에서 검색할 수 있는 문서들이 포함된 파일입니다.
- **loader.load()**를 호출하여 문서를 읽어들이고, 이를 documents라는 변수에 저장합니다. 이 변수는 나중에 벡터화(임베딩)할 때 사용됩니다.

In [ ]:
# 2. 문서 임베딩 및 인덱싱
# 문서를 벡터로 변환한 후, FAISS 벡터 저장소에 저장합니다.
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(documents, embeddings)

이 부분은 문서를 벡터로 변환하고 이를 인덱싱하는 단계입니다.

- OpenAIEmbeddings: 문서를 숫자 벡터로 변환하는데 사용됩니다. 임베딩은 텍스트의 의미를 벡터 공간에서 표현하는 방법으로, 유사성 검색에 중요한 역할을 합니다.
- FAISS.from_documents: documents 변수를 임베딩한 후, FAISS 인덱스를 생성합니다. 이 인덱스는 문서를 벡터로 변환한 후 빠르게 유사성을 계산하고 관련 문서를 검색하는 데 사용됩니다.

In [ ]:
# 3. Fine-tuned GPT 모델을 사용한 LLM 생성
# Fine-tuned된 GPT 모델을 사용할 수 있도록 설정합니다.
llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0.3)

이 부분은 GPT 모델을 설정하는 부분입니다.

- ChatOpenAI: OpenAI의 GPT 모델을 사용합니다. 여기서 model 파라미터로 Fine-tuned된 GPT 모델을 지정하고 있습니다. "gpt-4o-mini-2024-07-18"은 Fine-tuning된 GPT 모델의 ID입니다.
- temperature: 텍스트 생성을 제어하는 파라미터로, 값이 낮을수록 더 결정적인(deterministic) 결과를 생성하고, 값이 높을수록 생성되는 텍스트의 창의성(랜덤성)이 높아집니다.

In [ ]:
# 4. 문서 검색 개수 조정 (최대 5개의 문서 검색)
# 검색할 문서의 개수를 5개로 설정하여 모든 문서가 context에 반영될 수 있도록 설정합니다.
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

이 부분에서는 검색할 문서의 개수를 설정합니다.

- retriever: vector_store에서 as_retriever 메서드를 사용하여 문서 검색을 위한 리트리버(retriever)를 생성합니다.
- k 값: search_kwargs에서 k를 5로 설정하여, 사용자가 질문을 할 때 최대 5개의 문서가 검색되도록 설정합니다. 즉, 질문과 관련된 상위 5개의 문서를 검색하여 답변에 사용할 수 있도록 합니다.

In [ ]:
# 5. RAG 구현
# RetrievalQA 체인을 사용해 검색된 문서를 기반으로 LLM을 통해 답변을 생성합니다.
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,  # Fine-tuned된 GPT 모델.
    chain_type="stuff",  # 'stuff' 체인을 사용하여 검색된 문서들을 단일 입력으로 결합하여 처리.
    retriever=retriever,  # 검색할 문서의 수를 5개로 설정한 retriever.
    return_source_documents=True  # 검색된 문서를 포함하여 반환할지 여부를 지정
)

이 부분은 Retrieval-Augmented Generation (RAG) 체인을 구현하는 부분입니다.

- RetrievalQA: RetrievalQA 체인은 검색된 문서를 GPT 모델과 결합하여 질문에 대한 답변을 생성합니다.
- from_chain_type: 여기서는 "stuff" 체인 유형을 사용합니다. 이는 검색된 문서들을 모두 결합하여 하나의 텍스트로 처리하는 방식입니다.
- retriever: 위에서 생성한 retriever를 사용하여 문서 검색 작업을 처리합니다.
- return_source_documents=True: 검색된 문서를 응답에 포함하여 반환할지 여부를 지정합니다. True로 설정하면 검색된 문서들이 결과에 포함됩니다.

In [ ]:
# 6. 질문에 대한 답변 생성
query = "What is the impact of climate change on polar bears?"
result = qa_chain.invoke({"query": query})  # invoke 메서드를 사용하여 query 전달

# 7. 결과 출력
print(result["result"])

여기서는 질문에 대한 답변을 생성하는 단계입니다.

- query: 사용자가 묻고자 하는 질문을 정의합니다. 이 예시에서는 "기후 변화가 북극곰에 미치는 영향은 무엇인가요?"라는 질문을 사용하고 있습니다.
- qa_chain.invoke(): 이 메서드는 query를 입력받아 검색된 문서들을 사용하여 GPT 모델이 답변을 생성하게 합니다.

전체 코드

In [14]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA

# OpenAI API 키 설정
import os
os.environ["OPENAI_API_KEY"] = 'sk-proj-upOVy97VapEQl5kOiTQIT3BlbkFJlKBrtAnjDU5YMvWrqMzd'  # OpenAI API 키 입력

# 1. 문서 로드
# 텍스트 파일을 사용해 문서를 로드합니다.
loader = TextLoader('documents.txt')
documents = loader.load()

# 2. 문서 임베딩 및 인덱싱
# 문서를 벡터로 변환한 후, FAISS 벡터 저장소에 저장합니다.
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(documents, embeddings)

# 3. Fine-tuned GPT 모델을 사용한 LLM 생성
# Fine-tuned된 GPT 모델을 사용할 수 있도록 설정합니다.
llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0.3)

# 4. 문서 검색 개수 조정 (최대 5개의 문서 검색)
# 검색할 문서의 개수를 5개로 설정하여 모든 문서가 context에 반영될 수 있도록 설정합니다.
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

# 5. RAG 구현
# RetrievalQA 체인을 사용해 검색된 문서를 기반으로 LLM을 통해 답변을 생성합니다.
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,  # Fine-tuned된 GPT 모델.
    chain_type="stuff",  # 'stuff' 체인을 사용하여 검색된 문서들을 단일 입력으로 결합하여 처리.
    retriever=retriever,  # 검색할 문서의 수를 5개로 설정한 retriever.
    return_source_documents=True  # 검색된 문서를 포함하여 반환할지 여부를 지정
)

# 6. 질문에 대한 답변 생성
query = "What is the impact of climate change on polar bears?"
result = qa_chain.invoke({"query": query})  # invoke 메서드를 사용하여 query 전달

# 7. 결과 출력
print(result["result"])

Climate change has a significant impact on polar bears primarily through the loss of sea ice, which is essential for their hunting and survival. As global temperatures rise, sea ice is melting earlier and forming later each year, reducing the time polar bears have to hunt seals. This leads to decreased body condition, lower cub survival rates, and increased mortality.

Additionally, the retreating ice forces polar bears to travel greater distances to find food, resulting in increased energy expenditure, malnutrition, and a decline in reproductive success. Warmer temperatures also affect their denning areas, making it more difficult for pregnant females to find suitable sites for giving birth, which is crucial for the survival of their cubs.

As traditional food sources become less accessible, some polar bears have started scavenging on human waste or preying on seabirds and their eggs, but this does not provide the high-fat diet they need for long-term survival. If current trends in gr